# Robotics Foundations (H) - Lab 2

## ILOs
In this week's lab, you will understand what's behind the Baxter robot (Figure 1) simulation and how its kinematic transformations are maintained. The Baxter robot is an industrial robot built by Rethink Robotics. It has two arms and is intended for use in manufacturing production lines. In this lab, you will:

* Understand how ROS handles coordinate transformations and how this translates into a robot (ref. Lecture 3)
* Understand the Unified Robot Description Format (URDF) and XML macros (aka XACRO) (ref. Lecture 4)

![baxter_glasgow.png](imgs/baxter_glasgow.jpg)
<div style="text-align:center"><b>Figure 1: The Baxter robot</b><br></div>

## Coordinate Transforms in ROS
As in the lecture, coordinate frame transformations are fundamental to robotics. For articulated robot arms, you can deduce the forward and inverse kinematics to compute gripper poses as a function of joint angles or vice-versa. The latter is possible by multiplying transformations describing the current state of the robot. Sensor data, e.g. from cameras, can be merged into the robot's coordinate frame, and this data must be interpreted in terms of alternative frames (e.g. world frame or robot frame) to allow a robot to perceive and act within the environment. 

ROS provides a useful package for handling, maintaining and updating these transformations. This package is [tf](http://wiki.ros.org/tf) and offers command-line tools for inspecting transformations of a robot. TF also provides APIs in ROS's supported programming languages to manipulate and plan movements with robots. 

To find out how to use TF, let's start Gazebo and bring up a robot (Baxter) by typing in the terminal (you can find a terminal shortcut in the Desktop):

```bash
roslaunch baxter_gazebo baxter_world.launch
```

Baxter has several ROS topics being published, including the topic for transformations "tf". Press Ctrl+Enter to run the following command which will print information about the "tf" topic:

In [ ]:
%%bash
source ~/ros_ws/devel/setup.bash
rostopic info tf

To find more about `rostopic`, [go here!](http://wiki.ros.org/rostopic). You will find out that `rostopic` is a very neat tool for inspecting and getting message types of available topics within a ROS system. The `tf` topic uses messages of type [tf2_msgs/TFMessage](http://docs.ros.org/kinetic/api/tf2_msgs/html/msg/TFMessage.html) which consists of a variable-length array of a [stamped transform](http://docs.ros.org/kinetic/api/geometry_msgs/html/msg/TransformStamped.html) message. ROS does not employ the homogenous 4-by-4 matrix representation covered in the lecture but uses a 3-D vector (equivalent to the fourth column of a 4-by-4 transform for denoting translations) and a [quaternion](http://www.euclideanspace.com/maths/algebra/realNormedAlgebra/quaternions/). The quaternion representation is an alternative to homogenous matrices, have powerful mathematical properties, and avoid several problems while using homogeneous transformations and Euler angles. The quaternion's data structure is more efficient to carry pose information than the homogenous matrix representation. That is, homogenous transformations need 12 floating-point numbers (16 numbers but the last row is always a row vector equal to 0 and the 4th entry equal to 1) as opposed to 7 floating-point numbers when using quaternions. Transform messages also have timestamps, and they explicitly name the child frame and the parent frame (as text strings).

In ROS, orientations are commonly expressed as unit quaternions. **However, you do not need to worry about quaternions that much because TF provides functions to transform quaternions into Euler angles and even homogenous transformations!** At this point, it is sufficient to know that there is a correspondence between quaternions and rotation matrices (ROS functions to perform such conversions will be introduced later) and that there are corresponding mathematical operations for coordinate transformations with quaternions.

There can be (and typically are) many publishers to the tf topic. Each publisher expresses a transforming relationship, describing a named child frame with respect to a named parent frame - more about this in the following sections. You can now examine the output of the `tf` topic with:

In [ ]:
%%bash
source ~/ros_ws/devel/setup.bash
rostopic echo /tf -n 1

The above command shows all available transformations for Baxter. As stated above, each message has a child and parent frame, which defines the transformation between the given link names. Each link name is a reference frame within the robot. Baxter's frames are defined with respect to a "world reference frame", e.g. the 0, 0, 0 coordinate - the origin, and is always defined in the `world` frame in ROS and is the parent to all transformations. To verify this, run in a terminal:

``` bash
rosrun tf tf_echo /world /base
```

As you can observe, the transformation between these links is defined at the origin. Another topic of interest is `joint_states`. This topic displays the angular position of Baxter's joints, and this information feeds to TF to compose all transformations between reference frames. `joint_states` lives in a ROS node that maintains a record of the joint states of a robot by either updating values from the motor's sensor readings or in software when there is an open loop control system (i.e. no feedback from motors). To inspect the contents of `joint_states`, press Shift+Enter the following cell:

In [ ]:
%%bash
source ~/ros_ws/devel/setup.bash
rostopic echo /robot/joint_states -n 1

In Baxter, the `joint_states` topic is published in `/robot/joint_states`; the rationale is that topics should be declared within a namespace. In this case, `joint_states` is part of the `robot` namespace (if you run `rostopic list`, you will find out that topics about Baxter are declared using the `robot` namespace). It is recommended to use namespaces in ROS to follow good software engineering practices, and it is convenient when the complexity of the ROS system increases.

The above message list the names of Baxter's joints and then the state of each joint in the same order, e.g. the left finger joint of the left gripper, `l_gripper_l_finger_joint`, is `0.0208` radians. These values are needed to compute transformations for link poses for all links defined in Baxter. For instance, the transformation pf Baxter's `head` frame with respect to the `base` frame cannot be computed without knowing the value of the `head_pan` angular position (see list above). To find out the transformation from `base` to `head`, run in a terminal:

``` bash
rosrun tf tf_echo /base /head
```

A parent can have multiple children, but a child must have a unique parent, thus generating a hierarchical tree of geometric relationships with respect to this parent frame. In ROS, you can create a `tf_listener` which is typically started as an independent thread. This thread subscribes to the `tf` topic and assembles a kinematic tree from individual parent-child transform messages. Since the `tf_listener` incorporates all transformations, it can address specific queries, such as *where is Baxter's right-hand finger frame relative to Baxter's head?”* The reply to this query is a transform message that can be used to reconcile different frames. As long as a complete tree connecting frames is published, the transform listener can be used to transform all available kinematic, sensor, etc. data into a common reference frame, thus allowing to display the data from multiple sources in a common frame.

To practice the above, let's create a `tf_listener` ROS node. For this, we need to create a ROS package named `lab2_pkg` with `rospy` as a dependency inside `~/workspace/rf_ws` (if you forgot how to create ROS packages, check out Lab 1), create a new ROS node named `tf_listener.py` (e.g. `cd` into your package `src` directory and type `touch tf_listener.py`), and locate the file using the file manager (double-click "Home" in the Desktop) and double-click on the file to open it (alternatively, you can right-clicki on `tf_listener.py` and choose VSCode). Then type the following (do not copy-paste the following, aim to type it as it will help you to rationalise the code):

``` python
#!/usr/bin/env python  
import rospy
import math

# Importing TF to facilitate the task of receiving transformations
import tf2_ros

if __name__ == '__main__':
    rospy.init_node('baxter_tf_listener')

    # This creates a transform listener object; once created, it starts receiving
    # transformations using the /tf topic and buffers them up for up to 10 seconds.
    tfBuffer = tf2_ros.Buffer()
    listener = tf2_ros.TransformListener(tfBuffer)

    # This is where the magic happens, a query is passed to the listener for the
    # /base to /head transform by means of the lookupTransform fn. The arguments are
    # from "this frame" to "this frame" at "this specific time"
    # (if you pass "rospy.Time(0), the fn will give you the latest available transform 
    rate = rospy.Rate(1.0)
    while not rospy.is_shutdown():
        try:
            transformation = tfBuffer.lookup_transform('base', 'head', rospy.Time())
        except (tf2_ros.LookupException, tf2_ros.ConnectivityException, tf2_ros.ExtrapolationException):
            rate.sleep()
            continue

        rospy.loginfo("Translation: \n" + str(transformation.transform.translation))
        rospy.loginfo("Quaternion: \n" + str(transformation.transform.rotation))
        
        rate.sleep()
```

Don't forget to make the node executable with `chmod +x tf_listener.py`, `catkin_make` in your `rf_ws` and source your workspace (run `source devel/setup.bash` in `rf_ws`)! The above listener will print the values of the translation and rotation of the transformation between `base` and `head`, every second until you press Ctrl+C. So to run it, type:

```bash
rosrun lab2_pkg tf_listener.py
```

You have to wait for a few seconds to start seeing the output of your `tf_listener.py`. [Transforms3d](http://matthew-brett.github.io/transforms3d/) is a very useful Python module that you can use to convert ROS transformations (translation and quaternions) into homogenous matrices, rotation matrices, Roll-Pitch-Yaw angles, etc. This Python module is already installed in the virtual machine! So import the `quaternions` module as `from transforms3d import quaternions` and `numpy` in the `tf_listener.py` ROS node. Now, add the following code above `rate.sleep()` in line 33:

``` python
q = transformation.transform.rotation
quat = (q.x, q.y, q.z, q.w) 
rot_mat = quaternions.quat2mat(quat)
rospy.loginfo("Rotation matrix \n" + str(rot_mat))

H = numpy.eye(4)
trans = transformation.transform.translation
H[:3,:3] = rot_mat
H[:3,3] = [trans.x, trans.y, trans.z]
rospy.loginfo("Homogenous matrix: \n" + str(H))

new_quat = quaternions.mat2quat(rot_mat)
rospy.loginfo("New quaternion: " + str(new_quat))
```

and rerun the node (no need to `catkin_make` your workspace; however, if you open a new terminal, remember to source your workspace). The above code demonstrates how to manipulate TF messages into homogeneous coordinates. Feel free to inspect available functions in the [Transforms3d](http://matthew-brett.github.io/transforms3d/), they will become useful when you want to pick up an object using Baxter's grippper, and you need to orient the gripper to grasp the object successfully!

## URDF - Unified Robot Description Format
Until now, you have been using the Gazebo simulator. Indeed, ROS allows you to use 3D models of a robot - these models usually come from CAD drawings used when designing a robot. Gazebo and RViz employ the [**Unified Robot Description Format**](http://wiki.ros.org/urdf) (URDF) to assemble these CAD files and describe the structure of the robot. That is, URDF is an XML that describes a robot, its parts, its joints, dimensions, and so on. So every time you see a 3D robot in ROS, a URDF file is associated with it. For instance, Baxter's URDF model can be obtained as follows (same as last week lab):

In [ ]:
%%bash
source ~/ros_ws/devel/setup.bash
rosparam get /robot_description

URDF files have two key fields that describe the geometry of a robot: `links` and `joints`. As in the previous section, the parent of all links is `base`, and this name must be unique to URDF files:

``` xml
  <link name="base">
  </link>
  <link name="torso">
    <visual>
      <origin rpy="0 0 0" xyz="0 0 0"/>
      <geometry>
        <mesh filename="package://baxter_description/meshes/torso/base_link.DAE"/>
      </geometry>
      <material name="darkgray">
        <color rgba=".2 .2 .2 1"/>
      </material>
    </visual>
    <collision>
      <origin rpy="0 0 0" xyz="0 0 0"/>
      <geometry>
        <mesh filename="package://baxter_description/meshes/torso/base_link_collision.DAE"/>
      </geometry>
    </collision>
    <inertial>
      <origin rpy="0 0 0" xyz="0.000000 0.000000 0.000000"/>
      <mass value="35.336455"/>
      <inertia ixx="1.849155" ixy="-0.000354" ixz="-0.154188" iyy="1.662671" iyz="0.003292" izz="0.802239"/>
    </inertial>
  </link>
```

To define what you see in the simulator, we use the `visual` field in the above code. Inside the code, you can define the geometry (cylinder, box, sphere, or mesh), the material (colour or texture), and the origin. In this case, the 3D mesh is a DAE file - DAE files are based on the [XML COLLADA format](https://www.khronos.org/collada/), which is short for Collaborative Design Activity. The code for the joint comes after where it is defined as the name, which must be unique as well. Also, it is defined as the joint type (fixed, revolute, continuous, floating, or planar, ref Lecture 3), and the parent and the child links. For Baxter, the `torso` is a child of `base`, it is set to the origin and is fixed, as shown below:

``` xml
  <joint name="torso_t0" type="fixed">
    <origin rpy="0 0 0" xyz="0 0 0"/>
    <parent link="base"/>
    <child link="torso"/>
  </joint>
```

URDF files can become quite large, especially for sophisticated robots such as Baxter. Baxter's URDF file has 1851 lines of XML code, where it is defined the robot, simulation parameters, grippers, hardware limits, etc. In RF, you only need to understand the above XML elements to know which link refers to what.

You can also use RViz to find out the latter. So, run Baxter's simulation if you don't have it running and type in a terminal:

``` bash
rosrun rviz rviz
```

Then, select the `Add` button under the Displays sidebar. In the new window that pops up, scroll to `Robot Model`, and then hit ‘OK’ to add a visual representation of the robot’s current pose. You now need to set the `Fixed Frame` (under `Global Options` to `base`; otherwise you will get a white lump in the middle. In `RobotModel`, expand the tree, and you should see the default parameters; the critical setting here is *Robot Description* which should point to the `robot_description`.

![rviz.png](imgs/rviz.png)
<div style="text-align:center"><b>Figure 2: RViz</b><br></div>

As stated above, you can examine Baxter's links, joints names and states in RViz. To do this, select `Add` and scroll to `TF` and hit `OK` to add a TF visualisation of links. You can deactivate `RobotModel` to have a better view of the links and reference frames, as in the figure below. You can change the `Marker Scale` parameter to make the names more readable. You can also deactivate links, coordinate frames visualisation, etc.; feel free to play with the options as this will come very handy later in the course.

![rviz_tf.png](imgs/rviz_tf.png)
<div style="text-align:center"><b>Figure 3: Displaying TF plugin in RViz</b><br></div>

### XACRO - XML Macros
Baxter's URDF file has 1851 lines of code, and imagine adding cameras, sensors and other geometries in it. The file will become quite complex. To minimise its complexity, ROS has the XACRO package which is XML macros and helps in minimising the overall size. XACRO also makes it easier to read and maintain, and allows robotic developers to create modules and reutilise them to create repeated structures for other robotic applications.

To use XACRO, it is important to specify a namespace so that the file is parsed correctly. For instance, the namespace for Baxter in the XACRO file is `baxter`, as shown below (Press Ctrl+Enter to evaluate the cell):

In [ ]:
%%bash
cd ~/ros_ws/src/baxter/baxter/baxter_common/baxter_description/urdf
cat baxter.urdf.xacro | grep "robot name"

One of the most convenient features in XACRO is the ability to declare constant values. Without the use of xacro, it would be almost impossible to maintain changes if you had to change some values. For instance, the constant values defined in Baxter's XACRO file are:

In [ ]:
%%bash
cd ~/ros_ws/src/baxter/baxter/baxter_common/baxter_description/urdf
cat baxter.urdf.xacro | grep "default"

You can build up arbitrarily complex expressions in the `${}` construct using the four basic operations (`+`, `-`, `*`, `/`). Exponentiation and modulus are, however, not supported. To evaluate constants, use the `${}` construct to evaluate them. Macros are the most useful component of the xacro package and can allow you to separate each robot's component into a well-structured collection of xacro files. In Baxter's main XACRO file, you can see `xacro:include`. This XML tag will indicate the parser to include and read the given file, e.g.:

``` xml
<!-- Baxter Pedestal -->
<xacro:include filename="$(find baxter_description)/urdf/baxter_base/baxter_base.urdf.xacro">
    <xacro:arg name="gazebo" value="${gazebo}"/>
</xacro:include>

<!-- Left End Effector -->
<xacro:include filename="$(find baxter_description)/urdf/left_end_effector.urdf.xacro" />
```

The above shows how to include a XACRO file and how to evaluate constants. Now, let's take a look at `left_end_effector.urdf.xacro`:

In [ ]:
%%bash
cd ~/ros_ws/src/baxter/baxter/baxter_common/baxter_description/urdf
cat left_end_effector.urdf.xacro

The above file includes another xacro file with the macro for Baxter's electric gripper (parametrised with 9 values) and then "calls" the macro with the corresponding values for each parameter. To invoke the macro, you need to add the XML tag name right after `xacro`! The definition of this macro is shown below: 

In [ ]:
%%bash
cd ~/ros_ws/src/baxter/baxter/baxter_common/rethink_ee_description/urdf/electric_gripper
cat rethink_electric_gripper.xacro

**Note:** If you inspect different Baxter's xacro files, you will find out that macros are scarce and there are several hard-coded variables. Robotic companies usually rely on this to circumvent Open Source licences! Indeed, releasing XACRO/URDF files give away the mechanical design of a robot!

To use the `.xacro` file with RViz and Gazebo, you need to convert it to `.urdf`. To do this, you need to execute the following command inside the folder where the main Baxter's xacro file is located in order to compile it as an URDF file. So in a terminal, type:

``` bash
cd ~/ros_ws/src/baxter/baxter/baxter_common/baxter_description/urdf
xacro baxter.urdf.xacro > baxter_lab.urdf
```

and `baxter_lab.urdf` will be the same as the output you got when running `rosparam get /robot_description` above. You do not need to worry about compil command. `xacro` is invoked every time you start Baxter's simulation!

Later in RF labs, you will use XACRO to add a camera sensor, objects and other robotic elements within the simulation and RViz. For now, open the File Manager (there's a shortcut in the Desktop) and navigate to  `~/ros_ws/src/baxter/baxter/baxter_common/baxter_description/urdf`. Open `baxter.urdf.xacro` and `baxter_lab.urdf` and inspect its contents. Pay close attention to the size of the URDF and XACRO files!

That's all for this Lab! :)